In [282]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv(r"F:\my awesome streamlit front end\subAfricaDf.csv")
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [283]:
df.head()

,Year,Month,Day,Country,State,City,Region,Region_name,duration_of_incident,AttackType,...,Summary,Group,Latitude,Longitude,Target_type,Weapon_type,sub_weapon_type,Suicide,Casualities,Success
0,1970,1,11,Ethiopia,Unknown,Unknown,11,Sub-Saharan Africa,0,Unknown,...,NaN,Eritrean Liberation Front,NaN,NaN,Military,Unknown,NaN,0,1.0,1
1,1970,3,14,Ethiopia,Unknown,Unknown,11,Sub-Saharan Africa,1,Hostage Taking (Kidnapping),...,NaN,Eritrean Liberation Front,NaN,NaN,Journalists & Media,Unknown,NaN,0,0.0,1
2,1970,4,0,Ethiopia,Unknown,Unknown,11,Sub-Saharan Africa,1,Hostage Taking (Kidnapping),...,NaN,Eritrean Liberation Front,NaN,NaN,Government (Diplomatic),Unknown,NaN,0,0.0,1
3,1971,1,15,Zambia,Unknown,Unknown,11,Sub-Saharan Africa,0,Hostage Taking (Kidnapping),...,NaN,Revolutionary Committee of Mozambique (COREMO),NaN,NaN,Government (General),Unknown,NaN,0,1.0,1
4,1971,1,22,Ethiopia,Amhara,Bahar Dar,11,Sub-Saharan Africa,0,Hijacking,...,NaN,Eritrean Liberation Front,11.583513,37.397019,Airports & Aircraft,Unknown,NaN,0,0.0,1


In [284]:
df = df[["Country", "State", "AttackType", "Target_type", "Group","Weapon_type","Suicide","Success"]]
df.head()

,Country,State,AttackType,Target_type,Group,Weapon_type,Suicide,Success
0,Ethiopia,Unknown,Unknown,Military,Eritrean Liberation Front,Unknown,0,1
1,Ethiopia,Unknown,Hostage Taking (Kidnapping),Journalists & Media,Eritrean Liberation Front,Unknown,0,1
2,Ethiopia,Unknown,Hostage Taking (Kidnapping),Government (Diplomatic),Eritrean Liberation Front,Unknown,0,1
3,Zambia,Unknown,Hostage Taking (Kidnapping),Government (General),Revolutionary Committee of Mozambique (COREMO),Unknown,0,1
4,Ethiopia,Amhara,Hijacking,Airports & Aircraft,Eritrean Liberation Front,Unknown,0,1


In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17550 entries, 0 to 17549
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      17550 non-null  object
 1   State        17542 non-null  object
 2   AttackType   17550 non-null  object
 3   Target_type  17550 non-null  object
 4   Group        17550 non-null  object
 5   Weapon_type  17550 non-null  object
 6   Suicide      17550 non-null  int64 
 7   Success      17550 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 1.1+ MB


In [286]:
df = df.dropna()
df.isnull().sum()

Country        0
State          0
AttackType     0
Target_type    0
Group          0
Weapon_type    0
Suicide        0
Success        0
dtype: int64

In [287]:
df['Country'].value_counts()

Somalia                             4142
Nigeria                             3907
South Africa                        2016
Sudan                                967
Democratic Republic of the Congo     775
Kenya                                683
Burundi                              613
Mali                                 566
Angola                               499
Uganda                               394
Mozambique                           363
Cameroon                             332
Central African Republic             283
South Sudan                          225
Ethiopia                             190
Rwanda                               159
Namibia                              151
Niger                                146
Senegal                              118
Zimbabwe                             101
Sierra Leone                         101
Chad                                  91
Rhodesia                              83
Ivory Coast                           74
Zambia          

In [288]:
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

In [289]:
country_map = shorten_categories(df.Country.value_counts(), 50)
df['Country'] = df['Country'].map(country_map)
df.Country.value_counts()

Somalia                             4142
Nigeria                             3907
South Africa                        2016
Sudan                                967
Democratic Republic of the Congo     775
Kenya                                683
Burundi                              613
Mali                                 566
Angola                               499
Uganda                               394
Mozambique                           363
Other                                340
Cameroon                             332
Central African Republic             283
South Sudan                          225
Ethiopia                             190
Rwanda                               159
Namibia                              151
Niger                                146
Senegal                              118
Zimbabwe                             101
Sierra Leone                         101
Chad                                  91
Rhodesia                              83
Ivory Coast     

In [290]:
df.State.value_counts()

Banaadir          1613
Borno             1423
Gauteng            862
Lower Shebelle     579
Unknown            539
                  ... 
Asmara               1
Ali Sabieh           1
Cacheu               1
Lacs                 1
Western Lakes        1
Name: State, Length: 464, dtype: int64

In [291]:
state_map = shorten_categories(df.State.value_counts(), 100)
df['State'] = df['State'].map(state_map)
df.State.value_counts()

Other               5653
Banaadir            1613
Borno               1423
Gauteng              862
Lower Shebelle       579
Unknown              539
KwaZulu-Natal        513
North Kivu           413
Lower Juba           341
Bay                  328
North Darfur         300
Bujumbura Mairie     276
Hiiraan              272
Extreme-North        255
Yobe                 253
Gedo                 246
Benue                213
Northern             212
Adamawa              208
South Darfur         203
Western Cape         195
Plateau              188
Rivers               184
Kaduna               169
Delta                168
Kano                 154
Bari                 142
Mudug                139
Middle Shebelle      138
Orientale            138
Eastern Cape         133
Kidal                132
Central              130
Gao                  129
Timbuktu             127
Galguduud            123
Bayelsa              122
North Eastern        111
Bakool               110
Mandera              108


In [292]:
df.AttackType.unique()

array(['Unknown', 'Hostage Taking (Kidnapping)', 'Hijacking',
       'Assassination', 'Bombing/Explosion', 'Armed Assault',
       'Facility/Infrastructure Attack',
       'Hostage Taking (Barricade Incident)', 'Unarmed Assault'],
      dtype=object)

In [293]:
df.Target_type.unique()

array(['Military', 'Journalists & Media', 'Government (Diplomatic)',
       'Government (General)', 'Airports & Aircraft', 'Business',
       'Private Citizens & Property', 'Transportation',
       'Violent Political Party', 'Educational Institution', 'Police',
       'Utilities', 'Religious Figures/Institutions', 'Tourists',
       'Unknown', 'NGO', 'Telecommunication', 'Food or Water Supply',
       'Maritime', 'Terrorists/Non-State Militia', 'Other'], dtype=object)

In [294]:
df.Group.unique()

array(['Eritrean Liberation Front',
       'Revolutionary Committee of Mozambique (COREMO)',
       'Black September', 'Unknown', 'Irish Republican Army (IRA)',
       'National Liberation Front of Chad', 'Maskini Liberation Front',
       'Front for the Liberation of the French Somali Coast',
       "People's Revolutionary Party (Tanzania)",
       "South-West Africa People's Organization (SWAPO)",
       'Popular Front for the Liberation of Palestine (PFLP)',
       'Coup plotters', 'Tigray Peoples Liberation Front (TPLF)',
       'African National Congress (South Africa)',
       'Front for the Liberation of the Enclave of Cabinda (FLEC)',
       'Somali Guerrillas', 'Polisario Front',
       'Soviet and Cuban-backed rebels/mercenaries', 'Afar rebels',
       'Zimbabwe African Nationalist Union (ZANU)',
       "Zimbabwe African People's Union", 'Zimbabwe Guerrillas',
       "Zimbabwe People's Army (ZIPA)",
       'National Union for the Total Independence of Angola (UNITA)',
       

In [295]:
df.Group.value_counts()

Unknown                                     6055
Al-Shabaab                                  3288
Boko Haram                                  2418
African National Congress (South Africa)     606
Fulani extremists                            511
                                            ... 
Al-Gama'at al-Islamiyya (IG)                   1
National Redemption Front                      1
Rally of Democratic Forces (RAFD)              1
Tribal Clash                                   1
Rastas                                         1
Name: Group, Length: 492, dtype: int64

In [296]:
group_map = shorten_categories(df.Group.value_counts(), 50)
df['Group'] = df['Group'].map(group_map)
df.Group.value_counts()

Unknown                                                        6055
Al-Shabaab                                                     3288
Boko Haram                                                     2418
Other                                                          1893
African National Congress (South Africa)                        606
Fulani extremists                                               511
National Union for the Total Independence of Angola (UNITA)     432
Lord's Resistance Army (LRA)                                    322
Mozambique National Resistance Movement (MNR)                   311
Allied Democratic Forces (ADF)                                  216
Janjaweed                                                       213
Hutu extremists                                                 167
Sudan People's Liberation Movement in Opposition (SPLM-IO)      101
Democratic Front for the Liberation of Rwanda (FDLR)             99
Movement for the Emancipation of the Niger Delta

In [297]:
df.Group.unique()

array(['Other', 'Unknown',
       "South-West Africa People's Organization (SWAPO)",
       'African National Congress (South Africa)',
       'National Union for the Total Independence of Angola (UNITA)',
       'Mozambique National Resistance Movement (MNR)',
       'Inkatha Freedom Party (IFP)',
       'Movement of Democratic Forces of Casamance', 'Hutu extremists',
       'Revolutionary United Front (RUF)', "Lord's Resistance Army (LRA)",
       'Allied Democratic Forces (ADF)', 'Mayi Mayi', 'Fulani extremists',
       'Janjaweed',
       'Movement for the Emancipation of the Niger Delta (MEND)',
       'Democratic Front for the Liberation of Rwanda (FDLR)',
       'Muslim extremists', 'Al-Shabaab',
       'Al-Qaida in the Islamic Maghreb (AQIM)', 'Boko Haram',
       "Sudan People's Liberation Movement - North",
       'Movement for Oneness and Jihad in West Africa (MUJAO)',
       'Ansar al-Dine (Mali)', 'Anti-Balaka Militia',
       "Sudan People's Liberation Movement in Opposit

In [298]:
df.Weapon_type.unique()

array(['Unknown', 'Explosives', 'Firearms', 'Incendiary', 'Melee',
       'Sabotage Equipment', 'Chemical', 'Other', 'Fake Weapons',
       'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)',
       'Biological'], dtype=object)

In [299]:
df.Weapon_type.value_counts()

Firearms                                                                       7495
Explosives                                                                     6318
Unknown                                                                        2368
Incendiary                                                                      840
Melee                                                                           478
Other                                                                            19
Chemical                                                                         12
Sabotage Equipment                                                                7
Biological                                                                        3
Fake Weapons                                                                      1
Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)       1
Name: Weapon_type, dtype: int64

In [300]:
weapon_map = shorten_categories(df.Weapon_type.value_counts(), 50)
df['Weapon_type'] = df['Weapon_type'].map(weapon_map)
df.Weapon_type.value_counts()

Firearms      7495
Explosives    6318
Unknown       2368
Incendiary     840
Melee          478
Other           43
Name: Weapon_type, dtype: int64

In [301]:
df.Weapon_type.unique()

array(['Unknown', 'Explosives', 'Firearms', 'Incendiary', 'Melee',
       'Other'], dtype=object)

In [302]:
df.Suicide.unique()

array([0, 1], dtype=int64)

In [303]:
df.Success.unique()

array([1, 0], dtype=int64)

In [304]:
from sklearn.preprocessing import LabelEncoder
le_state = LabelEncoder()
df['State'] = le_state.fit_transform(df['State'])
df["State"].unique()
#le.classes_

array([37, 32, 15,  2, 21, 11, 30,  9, 38, 18, 31, 14, 22, 36,  0, 19,  8,
        3, 28, 17, 16,  4, 23, 12, 25,  5, 29, 35, 34, 13, 10,  1, 33,  7,
       27, 20, 26,  6, 39, 24])

In [305]:
le_country = LabelEncoder()
df['Country'] = le_country.fit_transform(df['Country'])
df["Country"].unique()

array([ 7, 27, 26, 15, 23,  5, 21,  9, 20, 24, 12, 14,  0, 16, 11, 25, 18,
       28,  4,  1, 13, 10, 17, 19,  3,  2,  8,  6, 22])

In [306]:
le_group = LabelEncoder()
df['Group'] = le_group.fit_transform(df['Group'])
df["Group"].unique()

array([22, 27, 24,  0, 20, 18, 10, 17,  9, 23, 13,  3, 14,  8, 12, 16,  7,
       19,  2,  1,  6, 25, 15,  4,  5, 26, 21, 11])

In [307]:
le_weapon = LabelEncoder()
df['Weapon_type'] = le_weapon.fit_transform(df['Weapon_type'])
df["Weapon_type"].unique()

array([5, 0, 1, 2, 3, 4])

In [308]:
le_attack = LabelEncoder()
df['AttackType'] = le_attack.fit_transform(df['AttackType'])
df["AttackType"].unique()

array([8, 6, 4, 1, 2, 0, 3, 5, 7])

In [309]:
le_target = LabelEncoder()
df['Target_type'] = le_target.fit_transform(df['Target_type'])
df["Target_type"].unique()

array([ 8,  6,  4,  5,  0,  1, 12, 17, 20,  2, 11, 19, 13, 16, 18,  9, 14,
        3,  7, 15, 10])

In [310]:
df.Success.value_counts()

1    16269
0     1273
Name: Success, dtype: int64

In [311]:
X = df.drop("Success", axis=1)
y = df["Success"]

In [312]:
X.head()

,Country,State,AttackType,Target_type,Group,Weapon_type,Suicide
0,7,37,8,8,22,5,0
1,7,37,6,6,22,5,0
2,7,37,6,4,22,5,0
3,27,37,6,5,22,5,0
4,7,32,4,0,22,5,0


In [313]:
y.values

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [314]:
y.value_counts()

1    16269
0     1273
Name: Success, dtype: int64

In [315]:
X_train,X_test,y_train,y_test=train_test_split(X,y.values,train_size=0.8,stratify = y.values,random_state = 2)

In [316]:
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [317]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[  49  206]
 [  67 3187]]
0.9222000569962953
              precision    recall  f1-score   support

           0       0.42      0.19      0.26       255
           1       0.94      0.98      0.96      3254

    accuracy                           0.92      3509
   macro avg       0.68      0.59      0.61      3509
weighted avg       0.90      0.92      0.91      3509



In [318]:
X

,Country,State,AttackType,Target_type,Group,Weapon_type,Suicide
0,7,37,8,8,22,5,0
1,7,37,6,6,22,5,0
2,7,37,6,4,22,5,0
3,27,37,6,5,22,5,0
4,7,32,4,0,22,5,0
...,...,...,...,...,...,...,...
17545,9,32,3,1,2,1,0
17546,14,7,3,12,6,5,0
17547,20,2,2,8,2,0,0
17548,20,1,2,8,2,0,0


In [319]:
X = np.array([["Ethiopia", 'Unknown','Hostage Taking (Kidnapping)','Journalists & Media','Boko Haram','Unknown',0 ]])
X

array([['Ethiopia', 'Unknown', 'Hostage Taking (Kidnapping)',
        'Journalists & Media', 'Boko Haram', 'Unknown', '0']],
      dtype='<U27')

In [320]:
X[:, 0] = le_country.transform(X[:,0])
X[:, 1] = le_state.transform(X[:,1])
X[:, 2] = le_attack.transform(X[:,2])
X[:, 3] = le_target.transform(X[:,3])
X[:, 4] = le_group.transform(X[:,4])
X[:, 5] = le_weapon.transform(X[:,5])
X = X.astype(float)
X

array([[ 7., 37.,  6.,  6.,  6.,  5.,  0.]])

In [321]:
prediction = classifier.predict(X)
print(prediction)

if (prediction[0] == 0):
    print('The attack is not successful')
else:
    print('The attack is successful')

[1]
The attack is successful


In [322]:
import pickle

In [323]:
data = {"model":classifier , "le_country": le_country, "le_state": le_state,"le_attack":le_attack,"le_target":le_target,"le_group":le_group,"le_weapon":le_weapon}
with open('AA_LLC.pkl', 'wb') as file:
    pickle.dump(data, file)

In [324]:
with open('AA_LLC.pkl', 'rb') as file:
    data = pickle.load(file)

classifier_loaded = data["model"]
le_country = data["le_country"]
le_state = data["le_state"]
le_attack = data["le_attack"]
le_target = data["le_target"]
le_group = data["le_group"]
le_weapon = data["le_weapon"]

In [325]:
prediction = classifier_loaded.predict(X)
print(prediction)

if (prediction[0] == 0):
    print('The attack is not successful')
else:
    print('The attack is successful')

[1]
The attack is successful
